In [12]:
import pandas as pd
import nltk
import re

In [3]:
ttest = pd.read_csv('train_data.csv')

ttest

,question,answer
0,Would I ever need credit card if my debit card...,Skimmers are most likely at gas station pumps....
1,Cheapest way to wire or withdraw money from US...,There is a number of cheaper online options th...
2,How do I go about finding an honest ethical f...,Large and wellknown companies are typically a ...
3,Why invest in becoming a landlord?,why does it make sense financially to buy prop...
4,What could be the cause of a extreme highlow p...,Often these types of trades fall into two diff...
...,...,...
12042,What percent of my salary should I save?,I disagree with the selected answer. Theres no...
12043,Why do people invest in mutual fund rather tha...,How on earth can you possibly know what is goi...
12044,What would happen if the Euro currency went bust?,Each country would have to go back to its own ...
12045,Are credit cards not viewed as credit until yo...,Theres a difference between missing a payment ...


In [4]:
ttest.isnull().sum()

question    0
answer      0
dtype: int64

# tokenization

In [9]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tshmacm1167/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tshmacm1167/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
#nltk.download('punkt')
#def preprocess_text(text):
   # text = re.sub(r'[^a-zA-Z\s]', '', text)
   # text = text.lower()
    #tokens = word_tokenize(text)
    #stop_words = set(stopwords.words('english'))
    #tokens = [word for word in tokens if word not in stop_words]
    #return tokens


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tshmacm1167/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    tokens = text.split()
    stop_words = set([
        'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you',
        'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself',
        'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them',
        'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this',
        'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been',
        'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing',
        'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
        'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between',
        'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to',
        'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again',
        'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how',
        'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some',
        'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too',
        'very', 'can', 'will', 'just', 'don', 'should', 'now'
    ])
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

In [27]:
ttest['question_tokens'] = ttest['question'].apply(preprocess_text)
ttest['answer_tokens'] = ttest['answer'].apply(preprocess_text)

In [29]:
pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.7/252.7 MB 702.2 kB/s eta 0:00:0000:0100:09
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 851.4 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 633.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 788.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.4/670.4 kB 918.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 954.2 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [30]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

questions = ttest['question'].astype(str).tolist()
answers = ttest['answer'].astype(str).tolist()

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(questions + answers)

question_sequences = tokenizer.texts_to_sequences(questions)
answer_sequences = tokenizer.texts_to_sequences(answers)

max_len_q = max(len(seq) for seq in question_sequences)
max_len_a = max(len(seq) for seq in answer_sequences)

padded_questions = pad_sequences(question_sequences, maxlen=max_len_q, padding='post')
padded_answers = pad_sequences(answer_sequences, maxlen=max_len_a, padding='post')